In [59]:
from selenium import webdriver # 제어가 가능한 Webdriver 크롬 브라우저
from selenium.webdriver.common.by import By # select를 위한 선택자(CCS,ID,CLASS,NAME, XPATH 등)
from selenium.webdriver.common.keys import Keys # 키보드에 있는 키를 사용하기 위한 모듈
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import math
import os
import datetime

driver = webdriver.Chrome()

In [60]:
url = 'https://www.saramin.co.kr/zf_user/jobs/list/job-category'
driver.get(url)

### 경력 선택(신입)
elm = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sp_main_wrapper"]/div[1]/div[1]/button')))
career_elm = driver.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[1]/div[1]/button')
career_elm.click()
career_elm = driver.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[1]/div[1]/div/div[1]/div[1]/label')
career_elm.click()
close_elm = driver.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[1]/div[1]/div/div[3]/button[2]')
close_elm.click()

### 학력 선택(무관)
elm = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sp_main_wrapper"]/div[1]/div[2]/button')))
edu_elm = driver.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[1]/div[2]/button')
edu_elm.click()
edu_elm = driver.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[1]/div[2]/div/div[1]/div/label')
edu_elm.click()

### 지역 선택(서울 전체 선택)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sp_main_wrapper"]/div[2]/ul/li[2]/button')))
loc_elm = driver.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[2]/ul/li[2]/button')
loc_elm.click()
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sp_area_lastDepth_101000"]/li[1]/div/label')))
loc_elm = driver.find_element(By.XPATH, '//*[@id="sp_area_lastDepth_101000"]/li[1]/div/label')
loc_elm.click()


### 직업 선택(IT개발데이터 전체선택)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sp_main_wrapper"]/div[2]/ul/li[1]/button/span[1]')))
job_elm = driver.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[2]/ul/li[1]/button/span[1]')
job_elm.click()
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sp_main_wrapper"]/div[2]/div/div[1]/div[2]/div[1]/button[6]')))
job_elm = driver.find_element(By.XPATH, '//*[@id="sp_main_wrapper"]/div[2]/div/div[1]/div[2]/div[1]/button[6]')
job_elm.click()
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sp_job_category_subDepth_2"]/div[1]/label')))
job_elm = driver.find_element(By.XPATH, '//*[@id="sp_job_category_subDepth_2"]/div[1]/label')
job_elm.click()



### 겁색하기 버튼 클릭
btn_elm = driver.find_element(By.ID, 'search_btn')
btn_elm.click()

In [61]:
## Chrome 드라이버 html문서 만들기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [62]:
co = [i.text for i in soup.select('.str_tit')]
co

['\n                            (주)머큐리                        ',
 '(주)머큐리 각 부문별 정규직 채용 (영업, 관리, 개발)',
 '\n                            (주)베이직테크                        ',
 '㈜베이직테크 기업연구소 연구원 모집',
 '\n                            (주)가우포토                        ',
 '촬영장비 쇼핑몰 MD, 오프라인 영업, AS수리담당 채용',
 '\n                            주식회사 데브스토리즈                        ',
 'QA/테스터 정규직/인턴직 채용(경력무관)',
 '\n                            (주)위메진                        ',
 '병원 경영 지원서비스 회사 파이썬 개발자 모집',
 '\n                            큐어스트림(주)                        ',
 'SW개발팀 경력직 채용(프론트앤드&Firmware&서비스기획)',
 '\n                            (주)피코이노베이션                        ',
 'JAVA개발, CS/운영, 약국영업, 물류,경영관리 채용',
 '\n                            (주)야놀자                        ',
 'Data Engineer - Infra',
 '\n                            (주)알피바이오                        ',
 '알피바이오 신입/경력 채용',
 '\n                            (주)아이엔지스토리                        ',
 '[INGSTORY] 브랜딩 디자이너 채용',
 '\n                        

In [63]:
def parser_info(soup):
    company = [re.sub(r'[\n\r()]', '', job.strip()) for i, job in enumerate(co) if i % 2 == 0]
    title = [job.strip() for i, job in enumerate(co) if i % 2 != 0]
    stacks = []
    for i in soup.select('.job_sector'):
        st= []
        for j in i.select('span'):
            st.append(j.text)
        stacks.append(st)
    loc = [i.text for i in soup.select('.work_place')]
    career = [i.text for i in soup.select('.career')]
    edu = [i.text for i in soup.select('.education')]
    last_date = [i.text for i in soup.select('.support_detail>.date')]
    input_date = [i.text for i in soup.select('.deadlines')]
    main_url = 'https://www.saramin.co.kr'
    link = [main_url + i['href'] for i in soup.select('.job_tit>.str_tit')]

    dic = {'companys' : company, 'titles' : title, 'stack' : stacks,
       'location' : loc, 'careers' : career, 'education' : edu,
       'last_date' : last_date, 'input_date' : input_date, 'links' : link}

    return pd.DataFrame(dic)

In [64]:
df = parser_info(soup)

In [65]:
tc_elm = driver.find_element(By.XPATH, '//*[@id="content"]/div[5]/div/div[1]/span/em')
total_count = int(tc_elm.text)
page_cnt = math.ceil(total_count / 50)
page_cnt
end_cnt = page_cnt%10
ten_cnt = 10

## 1. 데이터 갯수가 501개 미만일 때
## 2. 데이터 갯수가 501개 이상 ~ 1001개 미만일 때
## 3. 데이터 갯수가 1001개 이상일 때

In [66]:
if total_count <501:
    for i in range(2, page_cnt+1):
        page_elm = driver.find_element(By.CSS_SELECTOR, f'#default_list_wrap > div > button:nth-child({i})')
        print('page :', page_elm.text)
        page_elm.click()
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')            
        new_df = parser_info(soup)
        df = pd.concat([df, new_df])        
elif total_count<1001:
    for i in range(page_cnt//10):
        if i <1:          
            for j in range(2, 12):
                page_elm = driver.find_element(By.CSS_SELECTOR, f'#default_list_wrap > div > button:nth-child({j})')
                print('page :', page_elm.text)
                page_elm.click()
                time.sleep(2)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')            
                new_df = parser_info(soup)
                df = pd.concat([df, new_df])              
        else:        
            if page_cnt%10 == 0:
                end_cnt = ten_cnt
            for k in range(3, end_cnt+2):
                time.sleep(2)
                page_elm = driver.find_element(By.CSS_SELECTOR, f'#default_list_wrap > div > button:nth-child({k})')
                print('page :', page_elm.text)
                page_elm.click()                   
                new_df = parser_info(soup)
                df = pd.concat([df, new_df])             
else:
    for i in range(page_cnt//10):
        if i <1:          
            for j in range(2, 12):
                page_elm = driver.find_element(By.CSS_SELECTOR, f'#default_list_wrap > div > button:nth-child({j})')
                print('page :', page_elm.text)
                page_elm.click()
                time.sleep(2)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')            
                new_df = parser_info(soup)
                df = pd.concat([df, new_df])             
        elif i == (page_cnt//10) -1:    
            if total_count%10 == 0:
                end_cnt = ten_cnt
            for k in range(3, end_cnt+2):
                time.sleep(2)
                page_elm = driver.find_element(By.CSS_SELECTOR, f'#default_list_wrap > div > button:nth-child({k})')
                print('page :', page_elm.text)
                page_elm.click()                   
                new_df = parser_info(soup)
                df = pd.concat([df, new_df])              
        else:
            for l in range(3, 13):
                time.sleep(2)
                page_elm = driver.find_element(By.CSS_SELECTOR, f'#default_list_wrap > div > button:nth-child({l})')
                print('page :', page_elm.text)
                page_elm.click()                   
                new_df = parser_info(soup)
                df = pd.concat([df, new_df])               

page : 2
page : 3
page : 4
page : 5
page : 6
page : 7
page : 8
page : 9
page : 10
page : 다음
page : 12
page : 13
page : 14
page : 15
page : 16
page : 17
page : 18
page : 19
page : 20


In [67]:
today = datetime.date.today().strftime('%Y%m%d')
df.to_excel(f'./취업정보/취업정보_{today}.xlsx', index=False, engine='openpyxl')